# Novel Movie Studio Analysis
## Flatiron School DC Data Science Module 1 Project
#### Joe Buzzelli and Alphonso Woodbury
Presented 03.06.2020

## Data Cleaning

### Assumptions
* As production ramps up, the studio will be best served to focus on domestic distribution; international revenues will not be analyzed
* Movie trends change over time; we will exclude date for films released more than 5 years ago
* Viewer ratings and sentiment can be an indicator of engagement (further analysis)

#### Import modules and datasets

In [3]:
import pandas as pd
import json
import numpy as np
import datetime
import calendar

In [4]:
!pwd

/Users/algrhythm/Flatiron/Projects/DCDS-MOD1---MS-Movies/notebooks


#### Datasets used (please confirm path)

In [6]:
budgets = pd.read_csv('/Users/algrhythm/Flatiron/Projects/DCDS-MOD1---MS-Movies/data/raw/tn.movie_budgets.csv.gz')
titles = pd.read_csv('/Users/algrhythm/Flatiron/Projects/DCDS-MOD1---MS-Movies/data/raw/imdb.title.basics.csv.gz')

# Dataset: budgets (tn.movie_budgets.csv.gz)

In [7]:
budgets

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [8]:
#convert budget columns to ints, calculate domestic profits
budgets.applymap(lambda c: c.strip() if isinstance(c, str) else c)
budgets[['production_budget', 'domestic_gross',]] = budgets[['production_budget', 'domestic_gross']].replace('[\$,]', '', regex=True).astype(int)
budgets['domestic_profit'] = budgets['domestic_gross'] - budgets['production_budget']

In [9]:
#set datetime, filter records to include only those in the last 5 years
budgets['release_date'] = pd.to_datetime(budgets['release_date'], yearfirst=True)
cutoff = np.datetime64('2015-03-01')
budgets = budgets.loc[(budgets['release_date'] > cutoff)]

In [10]:
#create composite ID with yyyy of release and movie name
budgets['year'] = budgets['release_date'].apply(lambda y: y.year)
budgets['budgets_id'] =  budgets['movie'] + ' ' + budgets['year'].astype(str)

/Users/algrhythm/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/algrhythm/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
#select and rename columns
budgets = budgets[['budgets_id','movie','release_date','production_budget','domestic_gross','domestic_profit']]
budgets = budgets.rename(columns={"movie": "budgets_movie"})

In [12]:
#sort df and set index
budgets = budgets.sort_values(by='domestic_gross', ascending=False)
budgets = budgets.reset_index(drop=True)

In [13]:
budgets

,budgets_id,budgets_movie,release_date,production_budget,domestic_gross,domestic_profit
0,Star Wars Ep. VII: The Force Awakens 2015,Star Wars Ep. VII: The Force Awakens,2015-12-18,306000000,936662225,630662225
1,Black Panther 2018,Black Panther,2018-02-16,200000000,700059566,500059566
2,Avengers: Infinity War 2018,Avengers: Infinity War,2018-04-27,300000000,678815482,378815482
3,Jurassic World 2015,Jurassic World,2015-06-12,215000000,652270625,437270625
4,Star Wars Ep. VIII: The Last Jedi 2017,Star Wars Ep. VIII: The Last Jedi,2017-12-15,317000000,620181382,303181382
...,...,...,...,...,...,...
883,Dancin' It's On 2015,Dancin' It's On,2015-10-30,12000000,0,-12000000
884,Lords of London 2015,Lords of London,2015-09-01,3440000,0,-3440000
885,To Write Love On Her Arms 2015,To Write Love On Her Arms,2015-03-03,3400000,0,-3400000
886,The Rhythm Section 2019,The Rhythm Section,2019-11-22,50000000,0,-50000000


# Dataset: titles (imdb.title.basics.csv.gz)

In [14]:
titles

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,NaN


In [15]:
#strip leading, trailing spaces and ensure all titles are str
titles['primary_title'] = titles['primary_title'].astype('str')
titles.applymap(lambda c: c.strip() if isinstance(c, str) else c)

#create composite id
titles['titles_id'] = titles['primary_title'] + ' '  + titles['start_year'].astype(str)

#select and rename columns
titles = titles[['titles_id','primary_title','genres','tconst']]
titles = titles.rename(columns={"primary_title": "titles_movie"})
titles

,titles_id,titles_movie,genres,tconst
0,Sunghursh 2013,Sunghursh,"Action,Crime,Drama",tt0063540
1,One Day Before the Rainy Season 2019,One Day Before the Rainy Season,"Biography,Drama",tt0066787
2,The Other Side of the Wind 2018,The Other Side of the Wind,Drama,tt0069049
3,Sabse Bada Sukh 2018,Sabse Bada Sukh,"Comedy,Drama",tt0069204
4,The Wandering Soap Opera 2017,The Wandering Soap Opera,"Comedy,Drama,Fantasy",tt0100275
...,...,...,...,...
146139,Kuambil Lagi Hatiku 2019,Kuambil Lagi Hatiku,Drama,tt9916538
146140,Rodolpho Teóphilo - O Legado de um Pioneiro 2015,Rodolpho Teóphilo - O Legado de um Pioneiro,Documentary,tt9916622
146141,Dankyavar Danka 2013,Dankyavar Danka,Comedy,tt9916706
146142,6 Gunn 2017,6 Gunn,NaN,tt9916730


# Join budgets & titles dataframes

In [16]:
name_match = pd.DataFrame()
name_match['budgets_movie'] = budgets['budgets_movie']
name_match['budgets_id'] = budgets['budgets_id']
name_match['match'] = budgets['budgets_movie'].isin(titles['titles_movie']) 
name_match = name_match[name_match['match'] ==False]
name_match

,budgets_movie,budgets_id,match
0,Star Wars Ep. VII: The Force Awakens,Star Wars Ep. VII: The Force Awakens 2015,False
4,Star Wars Ep. VIII: The Last Jedi,Star Wars Ep. VIII: The Last Jedi 2017,False
15,Guardians of the Galaxy Vol 2,Guardians of the Galaxy Vol 2 2017,False
31,Dr. Seussâ The Grinch,Dr. Seussâ The Grinch 2018,False
42,Mission: ImpossibleâFallout,Mission: ImpossibleâFallout 2018,False
...,...,...,...
877,Perfectos Desconocidos,Perfectos Desconocidos 2017,False
880,UtÃ¸ya 22. juli,UtÃ¸ya 22. juli 2017,False
881,Pourquoi j'ai pas mangÃ© mon pÃ¨re,Pourquoi j'ai pas mangÃ© mon pÃ¨re 2015,False
883,Dancin' It's On,Dancin' It's On 2015,False


In [17]:
budgets.set_index('budgets_id', inplace=True)
titles.set_index('titles_id', inplace=True)

In [18]:
b_and_t = budgets.join(titles, how='left')
b_and_t = b_and_t[b_and_t['domestic_profit'] >= 0] #!!
b_and_t = b_and_t.sort_values(by='domestic_profit',ascending=False)
b_and_t = b_and_t[b_and_t['domestic_profit'] >= 0] #!!
b_and_t = b_and_t.reset_index(drop=True)
b_and_t

,budgets_movie,release_date,production_budget,domestic_gross,domestic_profit,titles_movie,genres,tconst
0,Star Wars Ep. VII: The Force Awakens,2015-12-18,306000000,936662225,630662225,NaN,NaN,NaN
1,Black Panther,2018-02-16,200000000,700059566,500059566,Black Panther,"Action,Adventure,Sci-Fi",tt1825683
2,Jurassic World,2015-06-12,215000000,652270625,437270625,Jurassic World,"Action,Adventure,Sci-Fi",tt0369610
3,Incredibles 2,2018-06-15,200000000,608581744,408581744,Incredibles 2,"Action,Adventure,Animation",tt3606756
4,Avengers: Infinity War,2018-04-27,300000000,678815482,378815482,Avengers: Infinity War,"Action,Adventure,Sci-Fi",tt4154756
...,...,...,...,...,...,...,...,...
357,The Best of Enemies,2019-04-05,10000000,10205616,205616,The Best of Enemies,"Biography,Drama,History",tt4807408
358,Secret in Their Eyes,2015-11-20,20000000,20180155,180155,Secret in Their Eyes,"Crime,Drama,Mystery",tt1741273
359,Compadres,2016-04-22,3000000,3127773,127773,Compadres,"Action,Comedy,Crime",tt3367294
360,Krisha,2016-03-18,30000,144822,114822,NaN,NaN,NaN


### Note on additional cleaning
We identified that some title and budget data did not merge due to mismatch movie titles; we identified these movies and programmatically inserted the correct data

In [19]:
movies_nan = b_and_t[b_and_t['titles_movie'].isnull()]
movies_nan.head(10)

,budgets_movie,release_date,production_budget,domestic_gross,domestic_profit,titles_movie,genres,tconst
0,Star Wars Ep. VII: The Force Awakens,2015-12-18,306000000,936662225,630662225,NaN,NaN,NaN
9,Star Wars Ep. VIII: The Last Jedi,2017-12-15,317000000,620181382,303181382,NaN,NaN,NaN
18,Dr. Seussâ The Grinch,2018-11-09,75000000,270620950,195620950,NaN,NaN,NaN
21,Guardians of the Galaxy Vol 2,2017-05-05,200000000,389813101,189813101,NaN,NaN,NaN
25,A Star is Born,2018-10-05,36000000,215288866,179288866,NaN,NaN,NaN
41,Split,2017-01-20,5000000,138141585,133141585,NaN,NaN,NaN
50,John Wick: Chapter 3 â Parabellum,2019-05-17,40000000,141744320,101744320,NaN,NaN,NaN
51,Daddyâs Home,2015-12-25,50000000,150357137,100357137,NaN,NaN,NaN
52,Spider-Man: Into The Spider-Verse 3D,2018-12-14,90000000,190173195,100173195,NaN,NaN,NaN
65,Donât Breathe,2016-08-26,10000000,89217875,79217875,NaN,NaN,NaN


In [20]:
b_and_t.at[0, 'genres'] = 'Action,Adventure,Fantasy'
b_and_t.at[0, 'tconst'] = 'tt2527338'

b_and_t.at[9, 'genres'] = 'Action,Adventure,Fantasy'
b_and_t.at[9, 'tconst'] = 'tt2527336'

b_and_t.at[18, 'genres'] = 'Animation,Comedy,Family'
b_and_t.at[18, 'tconst'] = 'tt2709692'

b_and_t.at[21, 'genres'] = 'Action,Adventure,Comedy'
b_and_t.at[21, 'tconst'] = 'tt3896198'

b_and_t.at[25, 'genres'] = 'Drama,Music,Romance'
b_and_t.at[25, 'tconst'] = 'tt1517451'

b_and_t.at[41, 'genres'] = 'Drama,Fantasy'
b_and_t.at[41, 'tconst'] = 'tt3315656'

In [21]:
b_and_t

,budgets_movie,release_date,production_budget,domestic_gross,domestic_profit,titles_movie,genres,tconst
0,Star Wars Ep. VII: The Force Awakens,2015-12-18,306000000,936662225,630662225,NaN,"Action,Adventure,Fantasy",tt2527338
1,Black Panther,2018-02-16,200000000,700059566,500059566,Black Panther,"Action,Adventure,Sci-Fi",tt1825683
2,Jurassic World,2015-06-12,215000000,652270625,437270625,Jurassic World,"Action,Adventure,Sci-Fi",tt0369610
3,Incredibles 2,2018-06-15,200000000,608581744,408581744,Incredibles 2,"Action,Adventure,Animation",tt3606756
4,Avengers: Infinity War,2018-04-27,300000000,678815482,378815482,Avengers: Infinity War,"Action,Adventure,Sci-Fi",tt4154756
...,...,...,...,...,...,...,...,...
357,The Best of Enemies,2019-04-05,10000000,10205616,205616,The Best of Enemies,"Biography,Drama,History",tt4807408
358,Secret in Their Eyes,2015-11-20,20000000,20180155,180155,Secret in Their Eyes,"Crime,Drama,Mystery",tt1741273
359,Compadres,2016-04-22,3000000,3127773,127773,Compadres,"Action,Comedy,Crime",tt3367294
360,Krisha,2016-03-18,30000,144822,114822,NaN,NaN,NaN


#### API data

In [22]:
api = [{'Title': 'Star Wars: Episode VII - The Force Awakens',
 'Year': '2015',
 'Rated': 'PG-13',
 'Released': '18 Dec 2015',
 'Runtime': '138 min',
 'Genre': 'Action, Adventure, Sci-Fi',
 'Director': 'J.J. Abrams',
 'Writer': 'Lawrence Kasdan, J.J. Abrams, Michael Arndt, George Lucas (based on characters created by)',
 'Actors': 'Harrison Ford, Mark Hamill, Carrie Fisher, Adam Driver',
 'Plot': "Three decades after the Empire's defeat, a new threat arises in the militant First Order. Defected stormtrooper Finn and the scavenger Rey are caught up in the Resistance's search for the missing Luke Skywalker.",
 'Language': 'English',
 'Country': 'USA',
 'Awards': 'Nominated for 5 Oscars. Another 63 wins & 125 nominations.',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BOTAzODEzNDAzMl5BMl5BanBnXkFtZTgwMDU1MTgzNzE@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.9/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '93%'},
  {'Source': 'Metacritic', 'Value': '81/100'}],
 'Metascore': '81',
 'imdbRating': '7.9',
 'imdbVotes': '826,625',
 'imdbID': 'tt2488496',
 'Type': 'movie',
 'DVD': '05 Apr 2016',
 'BoxOffice': '$936,658,640',
 'Production': 'Walt Disney Pictures',
 'Website': 'N/A',
 'Response': 'True'},
{'Title': 'Black Panther',
  'Year': '2018',
  'Rated': 'PG-13',
  'Released': '16 Feb 2018',
  'Runtime': '134 min',
  'Genre': 'Action, Adventure, Sci-Fi',
  'Director': 'Ryan Coogler',
  'Writer': 'Ryan Coogler, Joe Robert Cole, Stan Lee (based on the Marvel comics by), Jack Kirby (based on the Marvel Comics by)',
  'Actors': "Chadwick Boseman, Michael B. Jordan, Lupita Nyong'o, Danai Gurira",
  'Plot': "T'Challa, heir to the hidden but advanced kingdom of Wakanda, must step forward to lead his people into a new future and must confront a challenger from his country's past.",
  'Language': 'English, Swahili, Nama, Xhosa, Korean',
  'Country': 'USA',
  'Awards': 'Won 3 Oscars. Another 108 wins & 266 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMTg1MTY2MjYzNV5BMl5BanBnXkFtZTgwMTc4NTMwNDI@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.3/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '97%'},
   {'Source': 'Metacritic', 'Value': '88/100'}],
  'Metascore': '88',
  'imdbRating': '7.3',
  'imdbVotes': '565,951',
  'imdbID': 'tt1825683',
  'Type': 'movie',
  'DVD': '15 May 2018',
  'BoxOffice': '$501,105,037',
  'Production': 'Marvel Studios',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Jurassic World',
  'Year': '2015',
  'Rated': 'PG-13',
  'Released': '12 Jun 2015',
  'Runtime': '124 min',
  'Genre': 'Action, Adventure, Sci-Fi',
  'Director': 'Colin Trevorrow',
  'Writer': 'Rick Jaffa (screenplay by), Amanda Silver (screenplay by), Colin Trevorrow (screenplay by), Derek Connolly (screenplay by), Rick Jaffa (story by), Amanda Silver (story by), Michael Crichton (based on the characters created by)',
  'Actors': "Chris Pratt, Bryce Dallas Howard, Irrfan Khan, Vincent D'Onofrio",
  'Plot': 'A new theme park, built on the original site of Jurassic Park, creates a genetically modified hybrid dinosaur, the Indominus Rex, which escapes containment and goes on a killing spree.',
  'Language': 'English',
  'Country': 'USA',
  'Awards': '15 wins & 57 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BNzQ3OTY4NjAtNzM5OS00N2ZhLWJlOWUtYzYwZjNmOWRiMzcyXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.0/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '72%'},
   {'Source': 'Metacritic', 'Value': '59/100'}],
  'Metascore': '59',
  'imdbRating': '7.0',
  'imdbVotes': '556,312',
  'imdbID': 'tt0369610',
  'Type': 'movie',
  'DVD': '20 Oct 2015',
  'BoxOffice': '$528,757,749',
  'Production': 'Universal Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Incredibles 2',
  'Year': '2018',
  'Rated': 'PG',
  'Released': '15 Jun 2018',
  'Runtime': '118 min',
  'Genre': 'Animation, Action, Adventure, Comedy, Family, Sci-Fi',
  'Director': 'Brad Bird',
  'Writer': 'Brad Bird',
  'Actors': 'Craig T. Nelson, Holly Hunter, Sarah Vowell, Huck Milner',
  'Plot': 'The Incredibles hero family takes on a new mission, which involves a change in family roles: Bob Parr (Mr Incredible) must manage the house while his wife Helen (Elastigirl) goes out to save the world.',
  'Language': 'English',
  'Country': 'USA',
  'Awards': 'Nominated for 1 Oscar. Another 12 wins & 72 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMTEzNzY0OTg0NTdeQTJeQWpwZ15BbWU4MDU3OTg3MjUz._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.6/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '94%'},
   {'Source': 'Metacritic', 'Value': '80/100'}],
  'Metascore': '80',
  'imdbRating': '7.6',
  'imdbVotes': '225,556',
  'imdbID': 'tt3606756',
  'Type': 'movie',
  'DVD': '23 Oct 2018',
  'BoxOffice': 'N/A',
  'Production': 'Disney/Pixar',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Avengers: Infinity War',
  'Year': '2018',
  'Rated': 'PG-13',
  'Released': '27 Apr 2018',
  'Runtime': '149 min',
  'Genre': 'Action, Adventure, Sci-Fi',
  'Director': 'Anthony Russo, Joe Russo',
  'Writer': 'Christopher Markus (screenplay by), Stephen McFeely (screenplay by), Stan Lee (based on the Marvel comics by), Jack Kirby (based on the Marvel comics by), Joe Simon (Captain America created by), Jack Kirby (Captain America created by), Steve Englehart (Star-Lord created by), Steve Gan (Star-Lord created by), Bill Mantlo (Rocket Raccoon created by), Keith Giffen (Rocket Raccoon created by), Jim Starlin (Thanos,  Gamora and Drax created by), Stan Lee (Groot created by), Larry Lieber (Groot created by), Jack Kirby (Groot created by), Steve Englehart (Mantis created by), Don Heck (Mantis created by)',
  'Actors': 'Robert Downey Jr., Chris Hemsworth, Mark Ruffalo, Chris Evans',
  'Plot': 'The Avengers and their allies must be willing to sacrifice all in an attempt to defeat the powerful Thanos before his blitz of devastation and ruin puts an end to the universe.',
  'Language': 'English',
  'Country': 'USA',
  'Awards': 'Nominated for 1 Oscar. Another 44 wins & 70 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMjMxNjY2MDU1OV5BMl5BanBnXkFtZTgwNzY1MTUwNTM@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.5/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '85%'},
   {'Source': 'Metacritic', 'Value': '68/100'}],
  'Metascore': '68',
  'imdbRating': '8.5',
  'imdbVotes': '748,996',
  'imdbID': 'tt4154756',
  'Type': 'movie',
  'DVD': '14 Aug 2018',
  'BoxOffice': '$664,987,816',
  'Production': 'Walt Disney Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Beauty and the Beast',
  'Year': '2017',
  'Rated': 'PG',
  'Released': '17 Mar 2017',
  'Runtime': '129 min',
  'Genre': 'Family, Fantasy, Musical, Romance',
  'Director': 'Bill Condon',
  'Writer': 'Stephen Chbosky (screenplay by), Evan Spiliotopoulos (screenplay by), Linda Woolverton (based on the 1991 animated film "Beauty and the Beast" animation screenplay by)',
  'Actors': 'Emma Watson, Dan Stevens, Luke Evans, Josh Gad',
  'Plot': 'A selfish Prince is cursed to become a monster for the rest of his life, unless he learns to fall in love with a beautiful young woman he keeps prisoner.',
  'Language': 'English',
  'Country': 'USA',
  'Awards': 'Nominated for 2 Oscars. Another 15 wins & 71 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMTUwNjUxMTM4NV5BMl5BanBnXkFtZTgwODExMDQzMTI@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.1/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '71%'},
   {'Source': 'Metacritic', 'Value': '65/100'}],
  'Metascore': '65',
  'imdbRating': '7.1',
  'imdbVotes': '253,795',
  'imdbID': 'tt2771200',
  'Type': 'movie',
  'DVD': '06 Jun 2017',
  'BoxOffice': '$503,974,884',
  'Production': 'Walt Disney Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Rogue One: A Star Wars Story',
  'Year': '2016',
  'Rated': 'PG-13',
  'Released': '16 Dec 2016',
  'Runtime': '133 min',
  'Genre': 'Action, Adventure, Sci-Fi',
  'Director': 'Gareth Edwards',
  'Writer': 'Chris Weitz (screenplay by), Tony Gilroy (screenplay by), John Knoll (story by), Gary Whitta (story by), George Lucas (based on characters created by)',
  'Actors': 'Felicity Jones, Diego Luna, Alan Tudyk, Donnie Yen',
  'Plot': 'The daughter of an Imperial scientist joins the Rebel Alliance in a risky move to steal the Death Star plans.',
  'Language': 'English',
  'Country': 'USA',
  'Awards': 'Nominated for 2 Oscars. Another 24 wins & 80 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMjEwMzMxODIzOV5BMl5BanBnXkFtZTgwNzg3OTAzMDI@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.8/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '84%'},
   {'Source': 'Metacritic', 'Value': '65/100'}],
  'Metascore': '65',
  'imdbRating': '7.8',
  'imdbVotes': '519,590',
  'imdbID': 'tt3748528',
  'Type': 'movie',
  'DVD': '04 Apr 2017',
  'BoxOffice': '$532,171,696',
  'Production': 'Walt Disney Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Jumanji: Welcome to the Jungle',
  'Year': '2017',
  'Rated': 'PG-13',
  'Released': '20 Dec 2017',
  'Runtime': '119 min',
  'Genre': 'Action, Adventure, Comedy, Fantasy',
  'Director': 'Jake Kasdan',
  'Writer': 'Chris McKenna (screenplay by), Erik Sommers (screenplay by), Scott Rosenberg (screenplay by), Jeff Pinkner (screenplay by), Chris McKenna (screen story by), Chris Van Allsburg (based on the book "Jumanji" by), Greg Taylor (based on the film "Jumanji" screen story by), Chris Van Allsburg (based on the film "Jumanji" screen story by), Jonathan Hensleigh (based on the film "Jumanji" screenplay by), Greg Taylor (based on the film "Jumanji" screenplay by), Jim Strain (based on the film "Jumanji" by)',
  'Actors': 'Dwayne Johnson, Kevin Hart, Jack Black, Karen Gillan',
  'Plot': 'Four teenagers are sucked into a magical video game, and the only way they can escape is to work together to finish the game.',
  'Language': 'English',
  'Country': 'USA, India, Canada, UK, Australia, Germany',
  'Awards': '5 wins & 14 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BODQ0NDhjYWItYTMxZi00NTk2LWIzNDEtOWZiYWYxZjc2MTgxXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.9/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '76%'},
   {'Source': 'Metacritic', 'Value': '58/100'}],
  'Metascore': '58',
  'imdbRating': '6.9',
  'imdbVotes': '284,295',
  'imdbID': 'tt2283362',
  'Type': 'movie',
  'DVD': '20 Mar 2018',
  'BoxOffice': '$393,201,353',
  'Production': 'Columbia Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Deadpool',
  'Year': '2016',
  'Rated': 'R',
  'Released': '12 Feb 2016',
  'Runtime': '108 min',
  'Genre': 'Action, Adventure, Comedy, Sci-Fi',
  'Director': 'Tim Miller',
  'Writer': 'Rhett Reese, Paul Wernick',
  'Actors': 'Ryan Reynolds, Karan Soni, Ed Skrein, Michael Benyaer',
  'Plot': 'A wisecracking mercenary gets experimented on and becomes immortal but ugly, and sets out to track down the man who ruined his looks.',
  'Language': 'English',
  'Country': 'USA',
  'Awards': 'Nominated for 2 Golden Globes. Another 28 wins & 75 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BYzE5MjY1ZDgtMTkyNC00MTMyLThhMjAtZGI5OTE1NzFlZGJjXkEyXkFqcGdeQXVyNjU0OTQ0OTY@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.0/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '85%'},
   {'Source': 'Metacritic', 'Value': '65/100'}],
  'Metascore': '65',
  'imdbRating': '8.0',
  'imdbVotes': '857,421',
  'imdbID': 'tt1431045',
  'Type': 'movie',
  'DVD': '10 May 2016',
  'BoxOffice': '$328,674,489',
  'Production': '20th Century Fox',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Star Wars: Episode VIII - The Last Jedi',
  'Year': '2017',
  'Rated': 'PG-13',
  'Released': '15 Dec 2017',
  'Runtime': '152 min',
  'Genre': 'Action, Adventure, Fantasy, Sci-Fi',
  'Director': 'Rian Johnson',
  'Writer': 'Rian Johnson, George Lucas (based on characters created by)',
  'Actors': 'Mark Hamill, Carrie Fisher, Adam Driver, Daisy Ridley',
  'Plot': 'Rey develops her newly discovered abilities with the guidance of Luke Skywalker, who is unsettled by the strength of her powers. Meanwhile, the Resistance prepares for battle with the First Order.',
  'Language': 'English',
  'Country': 'USA',
  'Awards': 'Nominated for 4 Oscars. Another 23 wins & 89 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMjQ1MzcxNjg4N15BMl5BanBnXkFtZTgwNzgwMjY4MzI@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.0/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '91%'},
   {'Source': 'Metacritic', 'Value': '85/100'}],
  'Metascore': '85',
  'imdbRating': '7.0',
  'imdbVotes': '524,507',
  'imdbID': 'tt2527336',
  'Type': 'movie',
  'DVD': '27 Mar 2018',
  'BoxOffice': '$619,117,636',
  'Production': 'Walt Disney Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'The Secret Life of Pets',
  'Year': '2016',
  'Rated': 'PG',
  'Released': '08 Jul 2016',
  'Runtime': '87 min',
  'Genre': 'Animation, Adventure, Comedy, Family',
  'Director': 'Chris Renaud, Yarrow Cheney(co-director)',
  'Writer': 'Cinco Paul, Ken Daurio, Brian Lynch',
  'Actors': 'Louis C.K., Eric Stonestreet, Kevin Hart, Jenny Slate',
  'Plot': 'The quiet life of a terrier named Max is upended when his owner takes in Duke, a stray whom Max instantly dislikes.',
  'Language': 'English',
  'Country': 'USA, France, Japan',
  'Awards': '4 wins & 15 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMjIzMzA1OTkzNV5BMl5BanBnXkFtZTgwODE3MjM4NzE@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.5/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '73%'},
   {'Source': 'Metacritic', 'Value': '61/100'}],
  'Metascore': '61',
  'imdbRating': '6.5',
  'imdbVotes': '173,142',
  'imdbID': 'tt2709768',
  'Type': 'movie',
  'DVD': '06 Dec 2016',
  'BoxOffice': '$368,305,960',
  'Production': 'Universal Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'It',
  'Year': '2017',
  'Rated': 'R',
  'Released': '08 Sep 2017',
  'Runtime': '135 min',
  'Genre': 'Horror',
  'Director': 'Andy Muschietti',
  'Writer': 'Chase Palmer (screenplay by), Cary Joji Fukunaga (screenplay by), Gary Dauberman (screenplay by), Stephen King (based on the novel by)',
  'Actors': 'Jaeden Martell, Jeremy Ray Taylor, Sophia Lillis, Finn Wolfhard',
  'Plot': 'In the summer of 1989, a group of bullied kids band together to destroy a shape-shifting monster, which disguises itself as a clown and preys on the children of Derry, their small Maine town.',
  'Language': 'English',
  'Country': 'USA, Canada',
  'Awards': '8 wins & 45 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BZDVkZmI0YzAtNzdjYi00ZjhhLWE1ODEtMWMzMWMzNDA0NmQ4XkEyXkFqcGdeQXVyNzYzODM3Mzg@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.3/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '86%'},
   {'Source': 'Metacritic', 'Value': '69/100'}],
  'Metascore': '69',
  'imdbRating': '7.3',
  'imdbVotes': '420,881',
  'imdbID': 'tt1396484',
  'Type': 'movie',
  'DVD': '09 Jan 2018',
  'BoxOffice': '$326,898,358',
  'Production': 'Warner Bros. Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Finding Dory',
  'Year': '2016',
  'Rated': 'PG',
  'Released': '17 Jun 2016',
  'Runtime': '97 min',
  'Genre': 'Animation, Adventure, Comedy, Family',
  'Director': 'Andrew Stanton, Angus MacLane(co-director)',
  'Writer': 'Andrew Stanton (original story by), Andrew Stanton (screenplay by), Victoria Strouse (screenplay by), Bob Peterson (additional screenplay material by), Angus MacLane (additional story material by)',
  'Actors': "Ellen DeGeneres, Albert Brooks, Ed O'Neill, Kaitlin Olson",
  'Plot': 'Friendly but forgetful blue tang Dory begins a search for her long-lost parents, and everyone learns a few things about the real meaning of family along the way.',
  'Language': 'English, Indonesian',
  'Country': 'USA',
  'Awards': 'Nominated for 1 BAFTA Film Award. Another 17 wins & 45 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BNzg4MjM2NDQ4MV5BMl5BanBnXkFtZTgwMzk3MTgyODE@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.3/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '94%'},
   {'Source': 'Metacritic', 'Value': '77/100'}],
  'Metascore': '77',
  'imdbRating': '7.3',
  'imdbVotes': '224,264',
  'imdbID': 'tt2277860',
  'Type': 'movie',
  'DVD': '15 Nov 2016',
  'BoxOffice': '$486,292,984',
  'Production': 'Walt Disney Pictures/PIXAR',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Wonder Woman',
  'Year': '2017',
  'Rated': 'PG-13',
  'Released': '02 Jun 2017',
  'Runtime': '141 min',
  'Genre': 'Action, Adventure, Fantasy, Sci-Fi, War',
  'Director': 'Patty Jenkins',
  'Writer': 'Allan Heinberg (screenplay by), Zack Snyder (story by), Allan Heinberg (story by), Jason Fuchs (story by), William Moulton Marston (Wonder Woman created by)',
  'Actors': 'Gal Gadot, Chris Pine, Connie Nielsen, Robin Wright',
  'Plot': 'When a pilot crashes and tells of conflict in the outside world, Diana, an Amazonian warrior in training, leaves home to fight a war, discovering her full powers and true destiny.',
  'Language': 'English, German, Dutch, French, Spanish, Chinese, Greek,  Ancient (to 1453), North American Indian',
  'Country': 'USA, China, Hong Kong',
  'Awards': '24 wins & 70 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BNDFmZjgyMTEtYTk5MC00NmY0LWJhZjktOWY2MzI5YjkzODNlXkEyXkFqcGdeQXVyMDA4NzMyOA@@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.4/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '93%'},
   {'Source': 'Metacritic', 'Value': '76/100'}],
  'Metascore': '76',
  'imdbRating': '7.4',
  'imdbVotes': '518,205',
  'imdbID': 'tt0451279',
  'Type': 'movie',
  'DVD': '19 Sep 2017',
  'BoxOffice': '$412,400,625',
  'Production': 'Warner Bros. Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Minions',
  'Year': '2015',
  'Rated': 'PG',
  'Released': '10 Jul 2015',
  'Runtime': '91 min',
  'Genre': 'Animation, Adventure, Comedy, Family, Sci-Fi',
  'Director': 'Kyle Balda, Pierre Coffin',
  'Writer': 'Brian Lynch',
  'Actors': 'Sandra Bullock, Jon Hamm, Michael Keaton, Allison Janney',
  'Plot': 'Minions Stuart, Kevin, and Bob are recruited by Scarlet Overkill, a supervillain who, alongside her inventor husband Herb, hatches a plot to take over the world.',
  'Language': 'English, Spanish',
  'Country': 'USA',
  'Awards': 'Nominated for 1 BAFTA Film Award. Another 4 wins & 25 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMTg2MTMyMzU0M15BMl5BanBnXkFtZTgwOTU3ODk4NTE@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.4/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '55%'},
   {'Source': 'Metacritic', 'Value': '56/100'}],
  'Metascore': '56',
  'imdbRating': '6.4',
  'imdbVotes': '201,804',
  'imdbID': 'tt2293640',
  'Type': 'movie',
  'DVD': '08 Dec 2015',
  'BoxOffice': '$277,969,969',
  'Production': 'Universal Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Captain Marvel',
  'Year': '2019',
  'Rated': 'PG-13',
  'Released': '08 Mar 2019',
  'Runtime': '123 min',
  'Genre': 'Action, Adventure, Sci-Fi',
  'Director': 'Anna Boden, Ryan Fleck',
  'Writer': 'Anna Boden (screenplay by), Ryan Fleck (screenplay by), Geneva Robertson-Dworet (screenplay by), Nicole Perlman (story by), Meg LeFauve (story by), Anna Boden (story by), Ryan Fleck (story by), Geneva Robertson-Dworet (story by)',
  'Actors': 'Brie Larson, Samuel L. Jackson, Ben Mendelsohn, Jude Law',
  'Plot': "Carol Danvers becomes one of the universe's most powerful heroes when Earth is caught in the middle of a galactic war between two alien races.",
  'Language': 'English',
  'Country': 'USA, Australia',
  'Awards': '6 wins & 34 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMTE0YWFmOTMtYTU2ZS00ZTIxLWE3OTEtYTNiYzBkZjViZThiXkEyXkFqcGdeQXVyODMzMzQ4OTI@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.9/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '78%'},
   {'Source': 'Metacritic', 'Value': '64/100'}],
  'Metascore': '64',
  'imdbRating': '6.9',
  'imdbVotes': '394,382',
  'imdbID': 'tt4154664',
  'Type': 'movie',
  'DVD': '11 Jun 2019',
  'BoxOffice': 'N/A',
  'Production': 'Marvel Studios',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Jurassic World: Fallen Kingdom',
  'Year': '2018',
  'Rated': 'PG-13',
  'Released': '22 Jun 2018',
  'Runtime': '128 min',
  'Genre': 'Action, Adventure, Sci-Fi',
  'Director': 'J.A. Bayona',
  'Writer': 'Derek Connolly, Colin Trevorrow, Michael Crichton (based on characters created by)',
  'Actors': 'Chris Pratt, Bryce Dallas Howard, Rafe Spall, Justice Smith',
  'Plot': "When the island's dormant volcano begins roaring to life, Owen and Claire mount a campaign to rescue the remaining dinosaurs from this extinction-level event.",
  'Language': 'English, Russian',
  'Country': 'USA',
  'Awards': '4 wins & 25 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BNzIxMjYwNDEwN15BMl5BanBnXkFtZTgwMzk5MDI3NTM@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.2/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '48%'},
   {'Source': 'Metacritic', 'Value': '51/100'}],
  'Metascore': '51',
  'imdbRating': '6.2',
  'imdbVotes': '240,399',
  'imdbID': 'tt4881806',
  'Type': 'movie',
  'DVD': '18 Sep 2018',
  'BoxOffice': 'N/A',
  'Production': 'Universal Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Deadpool 2',
  'Year': '2018',
  'Rated': 'R',
  'Released': '18 May 2018',
  'Runtime': '119 min',
  'Genre': 'Action, Adventure, Comedy, Sci-Fi',
  'Director': 'David Leitch',
  'Writer': 'Rhett Reese, Paul Wernick, Ryan Reynolds',
  'Actors': 'Ryan Reynolds, Josh Brolin, Morena Baccarin, Julian Dennison',
  'Plot': 'Foul-mouthed mutant mercenary Wade Wilson (a.k.a. Deadpool), brings together a team of fellow mutant rogues to protect a young boy with supernatural abilities from the brutal, time-traveling cyborg Cable.',
  'Language': 'English, Cantonese, Spanish, Russian',
  'Country': 'USA',
  'Awards': '6 wins & 50 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BNjk1Njk3YjctMmMyYS00Y2I4LThhMzktN2U0MTMyZTFlYWQ5XkEyXkFqcGdeQXVyODM2ODEzMDA@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.7/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '84%'},
   {'Source': 'Metacritic', 'Value': '66/100'}],
  'Metascore': '66',
  'imdbRating': '7.7',
  'imdbVotes': '436,516',
  'imdbID': 'tt5463162',
  'Type': 'movie',
  'DVD': '21 Aug 2018',
  'BoxOffice': 'N/A',
  'Production': '20th Century Fox',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': "Dr. Seuss' The Grinch",
  'Year': '2018',
  'Rated': 'PG',
  'Released': '09 Nov 2018',
  'Runtime': '85 min',
  'Genre': 'Animation, Family, Fantasy',
  'Director': 'Yarrow Cheney, Scott Mosier',
  'Writer': 'Michael LeSieur (screenplay by), Tommy Swerdlow (screenplay by), Dr. Seuss (based on the book "How The Grinch Stole Christmas" by)',
  'Actors': 'Benedict Cumberbatch, Cameron Seely, Rashida Jones, Pharrell Williams',
  'Plot': 'A grumpy Grinch (Benedict Cumberbatch) plots to ruin Christmas for the village of Whoville.',
  'Language': 'English',
  'Country': 'France, Japan, USA',
  'Awards': '1 win & 18 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BYmE5Yjg0MzktYzgzMi00YTFiLWJjYTItY2M5MmI1ODI4MDY3XkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.3/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '59%'},
   {'Source': 'Metacritic', 'Value': '51/100'}],
  'Metascore': '51',
  'imdbRating': '6.3',
  'imdbVotes': '42,325',
  'imdbID': 'tt2709692',
  'Type': 'movie',
  'DVD': '05 Feb 2019',
  'BoxOffice': 'N/A',
  'Production': 'Universal Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Sing',
  'Year': '2016',
  'Rated': 'PG',
  'Released': '21 Dec 2016',
  'Runtime': '108 min',
  'Genre': 'Animation, Comedy, Family, Musical',
  'Director': 'Garth Jennings, Christophe Lourdelet(co-director)',
  'Writer': 'Garth Jennings',
  'Actors': 'Matthew McConaughey, Reese Witherspoon, Seth MacFarlane, Scarlett Johansson',
  'Plot': "In a city of humanoid animals, a hustling theater impresario's attempt to save his theater with a singing competition becomes grander than he anticipates even as its finalists find that their lives will never be the same.",
  'Language': 'English, Japanese, Ukrainian',
  'Country': 'Japan, USA',
  'Awards': 'Nominated for 2 Golden Globes. Another 3 wins & 23 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMTYzODYzODU2Ml5BMl5BanBnXkFtZTgwNTc1MTA2NzE@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.1/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '72%'},
   {'Source': 'Metacritic', 'Value': '59/100'}],
  'Metascore': '59',
  'imdbRating': '7.1',
  'imdbVotes': '123,309',
  'imdbID': 'tt3470600',
  'Type': 'movie',
  'DVD': '21 Mar 2017',
  'BoxOffice': '$270,322,940',
  'Production': 'Illumination Entertainment',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Zootopia',
  'Year': '2016',
  'Rated': 'PG',
  'Released': '04 Mar 2016',
  'Runtime': '108 min',
  'Genre': 'Animation, Adventure, Comedy, Crime, Family, Mystery',
  'Director': 'Byron Howard, Rich Moore, Jared Bush(co-director)',
  'Writer': 'Byron Howard (story by), Rich Moore (story by), Jared Bush (story by), Jim Reardon (story by), Josie Trinidad (story by), Phil Johnston (story by), Jennifer Lee (story by), Jared Bush (screenplay by), Phil Johnston (screenplay by)',
  'Actors': 'Ginnifer Goodwin, Jason Bateman, Idris Elba, Jenny Slate',
  'Plot': 'In a city of anthropomorphic animals, a rookie bunny cop and a cynical con artist fox must work together to uncover a conspiracy.',
  'Language': 'English',
  'Country': 'USA',
  'Awards': 'Won 1 Oscar. Another 46 wins & 70 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BOTMyMjEyNzIzMV5BMl5BanBnXkFtZTgwNzIyNjU0NzE@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.0/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '98%'},
   {'Source': 'Metacritic', 'Value': '78/100'}],
  'Metascore': '78',
  'imdbRating': '8.0',
  'imdbVotes': '404,133',
  'imdbID': 'tt2948356',
  'Type': 'movie',
  'DVD': '07 Jun 2016',
  'BoxOffice': '$341,264,012',
  'Production': 'Walt Disney Animation Studios',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Guardians of the Galaxy Vol. 2',
  'Year': '2017',
  'Rated': 'PG-13',
  'Released': '05 May 2017',
  'Runtime': '136 min',
  'Genre': 'Action, Adventure, Comedy, Sci-Fi',
  'Director': 'James Gunn',
  'Writer': 'James Gunn, Dan Abnett (based on the Marvel comics by), Andy Lanning (based on the Marvel comics by), Steve Englehart (Star-Lord created by), Steve Gan (Star-Lord created by), Jim Starlin (Gamora and Drax created by), Stan Lee (Groot created by), Larry Lieber (Groot created by), Jack Kirby (Groot created by), Bill Mantlo (Rocket Raccoon created by), Keith Giffen (Rocket Raccoon created by), Steve Gerber (Howard the Duck created by), Val Mayerik (Howard the Duck created by)',
  'Actors': 'Chris Pratt, Zoe Saldana, Dave Bautista, Vin Diesel',
  'Plot': "The Guardians struggle to keep together as a team while dealing with their personal family issues, notably Star-Lord's encounter with his father the ambitious celestial being Ego.",
  'Language': 'English',
  'Country': 'USA',
  'Awards': 'Nominated for 1 Oscar. Another 14 wins & 52 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BNjM0NTc0NzItM2FlYS00YzEwLWE0YmUtNTA2ZWIzODc2OTgxXkEyXkFqcGdeQXVyNTgwNzIyNzg@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.6/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '85%'},
   {'Source': 'Metacritic', 'Value': '67/100'}],
  'Metascore': '67',
  'imdbRating': '7.6',
  'imdbVotes': '522,765',
  'imdbID': 'tt3896198',
  'Type': 'movie',
  'DVD': '22 Aug 2017',
  'BoxOffice': '$389,804,217',
  'Production': 'Walt Disney Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Despicable Me 3',
  'Year': '2017',
  'Rated': 'PG',
  'Released': '30 Jun 2017',
  'Runtime': '89 min',
  'Genre': 'Animation, Adventure, Comedy, Family, Fantasy, Sci-Fi',
  'Director': 'Kyle Balda, Pierre Coffin, Eric Guillon(co-director)',
  'Writer': 'Cinco Paul, Ken Daurio',
  'Actors': 'Steve Carell, Kristen Wiig, Trey Parker, Miranda Cosgrove',
  'Plot': 'Gru (Steve Carell) meets his long-lost charming, cheerful, and more successful twin brother Dru (Steve Carell), who wants to team up with him for one last criminal heist.',
  'Language': 'English',
  'Country': 'USA',
  'Awards': '32 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BNjUyNzQ2MTg3Ml5BMl5BanBnXkFtZTgwNzE4NDM3MTI@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.3/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '58%'},
   {'Source': 'Metacritic', 'Value': '49/100'}],
  'Metascore': '49',
  'imdbRating': '6.3',
  'imdbVotes': '102,071',
  'imdbID': 'tt3469046',
  'Type': 'movie',
  'DVD': '05 Dec 2017',
  'BoxOffice': '$264,194,540',
  'Production': 'Universal Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'The Jungle Book',
  'Year': '2016',
  'Rated': 'PG',
  'Released': '15 Apr 2016',
  'Runtime': '106 min',
  'Genre': 'Adventure, Drama, Family, Fantasy',
  'Director': 'Jon Favreau',
  'Writer': 'Justin Marks (screenplay by), Rudyard Kipling (based on the books by)',
  'Actors': 'Neel Sethi, Bill Murray, Ben Kingsley, Idris Elba',
  'Plot': 'After a threat from the tiger Shere Khan forces him to flee the jungle, a man-cub named Mowgli embarks on a journey of self discovery with the help of panther Bagheera and free-spirited bear Baloo.',
  'Language': 'English',
  'Country': 'UK, USA',
  'Awards': 'Won 1 Oscar. Another 32 wins & 55 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMTc3NTUzNTI4MV5BMl5BanBnXkFtZTgwNjU0NjU5NzE@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.4/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '94%'},
   {'Source': 'Metacritic', 'Value': '77/100'}],
  'Metascore': '77',
  'imdbRating': '7.4',
  'imdbVotes': '248,775',
  'imdbID': 'tt3040964',
  'Type': 'movie',
  'DVD': '30 Aug 2016',
  'BoxOffice': '$363,995,937',
  'Production': 'Walt Disney Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Inside Out',
  'Year': '2015',
  'Rated': 'PG',
  'Released': '19 Jun 2015',
  'Runtime': '95 min',
  'Genre': 'Animation, Adventure, Comedy, Drama, Family, Fantasy',
  'Director': 'Pete Docter, Ronnie Del Carmen(co-director)',
  'Writer': 'Pete Docter (original story by), Ronnie Del Carmen (original story by), Pete Docter (screenplay by), Meg LeFauve (screenplay by), Josh Cooley (screenplay by), Michael Arndt (additional story material by), Simon Rich (additional story material by), Bill Hader (additional dialogue by), Amy Poehler (additional dialogue by)',
  'Actors': 'Amy Poehler, Phyllis Smith, Richard Kind, Bill Hader',
  'Plot': 'After young Riley is uprooted from her Midwest life and moved to San Francisco, her emotions - Joy, Fear, Anger, Disgust and Sadness - conflict on how best to navigate a new city, house, and school.',
  'Language': 'English',
  'Country': 'USA',
  'Awards': 'Won 1 Oscar. Another 97 wins & 113 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BOTgxMDQwMDk0OF5BMl5BanBnXkFtZTgwNjU5OTg2NDE@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.2/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '98%'},
   {'Source': 'Metacritic', 'Value': '94/100'}],
  'Metascore': '94',
  'imdbRating': '8.2',
  'imdbVotes': '567,084',
  'imdbID': 'tt2096673',
  'Type': 'movie',
  'DVD': '03 Nov 2015',
  'BoxOffice': '$264,317,903',
  'Production': 'Disney/Pixar',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'A Star Is Born',
  'Year': '2018',
  'Rated': 'R',
  'Released': '05 Oct 2018',
  'Runtime': '136 min',
  'Genre': 'Drama, Music, Romance',
  'Director': 'Bradley Cooper',
  'Writer': 'Eric Roth (screenplay by), Bradley Cooper (screenplay by), Will Fetters (screenplay by), Moss Hart (based on the 1954 screenplay by), John Gregory Dunne (based on the 1976 screenplay by), Joan Didion (based on the 1976 screenplay by), Frank Pierson (based on the 1976 screenplay by), William A. Wellman (based on a story by), Robert Carson (based on a story by)',
  'Actors': 'Lady Gaga, Bradley Cooper, Sam Elliott, Andrew Dice Clay',
  'Plot': 'A musician helps a young singer find fame as age and alcoholism send his own career into a downward spiral.',
  'Language': 'English',
  'Country': 'USA',
  'Awards': 'Won 1 Oscar. Another 92 wins & 272 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BNmE5ZmE3OGItNTdlNC00YmMxLWEzNjctYzAwOGQ5ODg0OTI0XkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.7/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '90%'},
   {'Source': 'Metacritic', 'Value': '88/100'}],
  'Metascore': '88',
  'imdbRating': '7.7',
  'imdbVotes': '295,859',
  'imdbID': 'tt1517451',
  'Type': 'movie',
  'DVD': '19 Feb 2019',
  'BoxOffice': 'N/A',
  'Production': 'Warner Bros. Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Aquaman',
  'Year': '2018',
  'Rated': 'PG-13',
  'Released': '21 Dec 2018',
  'Runtime': '143 min',
  'Genre': 'Action, Adventure, Fantasy, Sci-Fi',
  'Director': 'James Wan',
  'Writer': 'David Leslie Johnson-McGoldrick (screenplay by), Will Beall (screenplay by), Geoff Johns (story by), James Wan (story by), Will Beall (story by), Paul Norris (Aquaman created by), Mort Weisinger (Aquaman created by)',
  'Actors': 'Jason Momoa, Amber Heard, Willem Dafoe, Patrick Wilson',
  'Plot': 'Arthur Curry, the human-born heir to the underwater kingdom of Atlantis, goes on a quest to prevent a war between the worlds of ocean and land.',
  'Language': 'English, Russian, Maori, Italian',
  'Country': 'USA, Australia',
  'Awards': '1 win & 37 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BOTk5ODg0OTU5M15BMl5BanBnXkFtZTgwMDQ3MDY3NjM@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.0/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '66%'},
   {'Source': 'Metacritic', 'Value': '55/100'}],
  'Metascore': '55',
  'imdbRating': '7.0',
  'imdbVotes': '323,678',
  'imdbID': 'tt1477834',
  'Type': 'movie',
  'DVD': 'N/A',
  'BoxOffice': 'N/A',
  'Production': 'Warner Bros. Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Get Out',
  'Year': '2017',
  'Rated': 'R',
  'Released': '24 Feb 2017',
  'Runtime': '104 min',
  'Genre': 'Horror, Mystery, Thriller',
  'Director': 'Jordan Peele',
  'Writer': 'Jordan Peele',
  'Actors': 'Daniel Kaluuya, Allison Williams, Catherine Keener, Bradley Whitford',
  'Plot': "A young African-American visits his white girlfriend's parents for the weekend, where his simmering uneasiness about their reception of him eventually reaches a boiling point.",
  'Language': 'English',
  'Country': 'USA, Japan',
  'Awards': 'Won 1 Oscar. Another 151 wins & 199 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMjUxMDQwNjcyNl5BMl5BanBnXkFtZTgwNzcwMzc0MTI@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.7/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '98%'},
   {'Source': 'Metacritic', 'Value': '84/100'}],
  'Metascore': '84',
  'imdbRating': '7.7',
  'imdbVotes': '442,130',
  'imdbID': 'tt5052448',
  'Type': 'movie',
  'DVD': '23 May 2017',
  'BoxOffice': '$175,428,355',
  'Production': 'Universal Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'A Quiet Place',
  'Year': '2018',
  'Rated': 'PG-13',
  'Released': '06 Apr 2018',
  'Runtime': '90 min',
  'Genre': 'Drama, Horror, Sci-Fi',
  'Director': 'John Krasinski',
  'Writer': 'Bryan Woods (screenplay by), Scott Beck (screenplay by), John Krasinski (screenplay by), Bryan Woods (story by), Scott Beck (story by)',
  'Actors': 'Emily Blunt, John Krasinski, Millicent Simmonds, Noah Jupe',
  'Plot': 'In a post-apocalyptic world, a family is forced to live in silence while hiding from monsters with ultra-sensitive hearing.',
  'Language': 'English, American Sign Language',
  'Country': 'USA',
  'Awards': 'Nominated for 1 Oscar. Another 33 wins & 108 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMjI0MDMzNTQ0M15BMl5BanBnXkFtZTgwMTM5NzM3NDM@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.5/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '95%'},
   {'Source': 'Metacritic', 'Value': '82/100'}],
  'Metascore': '82',
  'imdbRating': '7.5',
  'imdbVotes': '350,809',
  'imdbID': 'tt6644200',
  'Type': 'movie',
  'DVD': '10 Jul 2018',
  'BoxOffice': 'N/A',
  'Production': 'Paramount Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Furious 7',
  'Year': '2015',
  'Rated': 'PG-13',
  'Released': '03 Apr 2015',
  'Runtime': '137 min',
  'Genre': 'Action, Adventure, Thriller',
  'Director': 'James Wan',
  'Writer': 'Chris Morgan, Gary Scott Thompson (characters)',
  'Actors': 'Vin Diesel, Paul Walker, Jason Statham, Michelle Rodriguez',
  'Plot': 'Deckard Shaw seeks revenge against Dominic Toretto and his family for his comatose brother.',
  'Language': 'English, Thai, Arabic, Spanish',
  'Country': 'USA, China, Japan, Canada, United Arab Emirates',
  'Awards': 'Nominated for 1 Golden Globe. Another 33 wins & 34 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMTQxOTA2NDUzOV5BMl5BanBnXkFtZTgwNzY2MTMxMzE@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.2/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '81%'},
   {'Source': 'Metacritic', 'Value': '67/100'}],
  'Metascore': '67',
  'imdbRating': '7.2',
  'imdbVotes': '344,990',
  'imdbID': 'tt2820852',
  'Type': 'movie',
  'DVD': '15 Sep 2015',
  'BoxOffice': '$316,986,481',
  'Production': 'Universal Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Bohemian Rhapsody',
  'Year': '2018',
  'Rated': 'PG-13',
  'Released': '02 Nov 2018',
  'Runtime': '134 min',
  'Genre': 'Biography, Drama, Music',
  'Director': 'Bryan Singer',
  'Writer': 'Anthony McCarten (story by), Peter Morgan (story by), Anthony McCarten (screenplay by)',
  'Actors': 'Rami Malek, Lucy Boynton, Gwilym Lee, Ben Hardy',
  'Plot': 'The story of the legendary British rock band Queen and lead singer Freddie Mercury, leading up to their famous performance at Live Aid (1985).',
  'Language': 'English, British Sign Language',
  'Country': 'UK, USA',
  'Awards': 'Won 4 Oscars. Another 37 wins & 76 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMTA2NDc3Njg5NDVeQTJeQWpwZ15BbWU4MDc1NDcxNTUz._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.0/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '61%'},
   {'Source': 'Metacritic', 'Value': '49/100'}],
  'Metascore': '49',
  'imdbRating': '8.0',
  'imdbVotes': '409,760',
  'imdbID': 'tt1727824',
  'Type': 'movie',
  'DVD': '12 Feb 2019',
  'BoxOffice': 'N/A',
  'Production': '20th Century Fox',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Spider-Man: Homecoming',
  'Year': '2017',
  'Rated': 'PG-13',
  'Released': '07 Jul 2017',
  'Runtime': '133 min',
  'Genre': 'Action, Adventure, Sci-Fi',
  'Director': 'Jon Watts',
  'Writer': 'Jonathan Goldstein (screenplay by), John Francis Daley (screenplay by), Jon Watts (screenplay by), Christopher Ford (screenplay by), Chris McKenna (screenplay by), Erik Sommers (screenplay by), Jonathan Goldstein (screen story by), John Francis Daley (screen story by), Stan Lee (based on the Marvel comic book by), Steve Ditko (based on the Marvel comic book by), Joe Simon (Captain America created by), Jack Kirby (Captain America created by)',
  'Actors': 'Tom Holland, Michael Keaton, Robert Downey Jr., Marisa Tomei',
  'Plot': 'Peter Parker balances his life as an ordinary high school student in Queens with his superhero alter-ego Spider-Man, and finds himself on the trail of a new menace prowling the skies of New York City.',
  'Language': 'English, Spanish',
  'Country': 'USA',
  'Awards': '7 wins & 9 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BNTk4ODQ1MzgzNl5BMl5BanBnXkFtZTgwMTMyMzM4MTI@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.4/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '92%'},
   {'Source': 'Metacritic', 'Value': '73/100'}],
  'Metascore': '73',
  'imdbRating': '7.4',
  'imdbVotes': '470,359',
  'imdbID': 'tt2250912',
  'Type': 'movie',
  'DVD': '17 Oct 2017',
  'BoxOffice': '$334,166,825',
  'Production': 'Sony Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Captain America: Civil War',
  'Year': '2016',
  'Rated': 'PG-13',
  'Released': '06 May 2016',
  'Runtime': '147 min',
  'Genre': 'Action, Adventure, Sci-Fi',
  'Director': 'Anthony Russo, Joe Russo',
  'Writer': 'Christopher Markus (screenplay by), Stephen McFeely (screenplay by), Joe Simon (based on the Marvel comics by), Jack Kirby (based on the Marvel comics by)',
  'Actors': 'Chris Evans, Robert Downey Jr., Scarlett Johansson, Sebastian Stan',
  'Plot': "Political involvement in the Avengers' affairs causes a rift between Captain America and Iron Man.",
  'Language': 'English, German, Xhosa, Russian, Romanian, Hindi',
  'Country': 'USA',
  'Awards': '16 wins & 68 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMjQ0MTgyNjAxMV5BMl5BanBnXkFtZTgwNjUzMDkyODE@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.8/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '91%'},
   {'Source': 'Metacritic', 'Value': '75/100'}],
  'Metascore': '75',
  'imdbRating': '7.8',
  'imdbVotes': '619,898',
  'imdbID': 'tt3498820',
  'Type': 'movie',
  'DVD': '13 Sep 2016',
  'BoxOffice': '$408,080,554',
  'Production': 'Walt Disney Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Pitch Perfect 2',
  'Year': '2015',
  'Rated': 'PG-13',
  'Released': '15 May 2015',
  'Runtime': '115 min',
  'Genre': 'Comedy, Music',
  'Director': 'Elizabeth Banks',
  'Writer': 'Kay Cannon, Mickey Rapkin (based on the book by)',
  'Actors': 'Anna Kendrick, Rebel Wilson, Hailee Steinfeld, Brittany Snow',
  'Plot': 'After a humiliating command performance at The Kennedy Center, the Barden Bellas enter an international competition that no American group has ever won in order to regain their status and right to perform.',
  'Language': 'English, German',
  'Country': 'USA',
  'Awards': '9 wins & 16 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMzk4OTM2NzMzNl5BMl5BanBnXkFtZTgwNTAzNDE0NDE@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.4/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '66%'},
   {'Source': 'Metacritic', 'Value': '63/100'}],
  'Metascore': '63',
  'imdbRating': '6.4',
  'imdbVotes': '135,698',
  'imdbID': 'tt2848292',
  'Type': 'movie',
  'DVD': '22 Sep 2015',
  'BoxOffice': '$165,744,048',
  'Production': 'Universal Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Us',
  'Year': '2019',
  'Rated': 'R',
  'Released': '22 Mar 2019',
  'Runtime': '116 min',
  'Genre': 'Horror, Mystery, Thriller',
  'Director': 'Jordan Peele',
  'Writer': 'Jordan Peele',
  'Actors': "Lupita Nyong'o, Winston Duke, Elisabeth Moss, Tim Heidecker",
  'Plot': "A family's serene beach vacation turns to chaos when their doppelgängers appear and begin to terrorize them.",
  'Language': 'English',
  'Country': 'USA, China, Japan',
  'Awards': '65 wins & 96 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BZTliNWJhM2YtNDc1MC00YTk1LWE2MGYtZmE4M2Y5ODdlNzQzXkEyXkFqcGdeQXVyMzY0MTE3NzU@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.9/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '93%'},
   {'Source': 'Metacritic', 'Value': '81/100'}],
  'Metascore': '81',
  'imdbRating': '6.9',
  'imdbVotes': '182,610',
  'imdbID': 'tt6857112',
  'Type': 'movie',
  'DVD': '18 Jun 2019',
  'BoxOffice': 'N/A',
  'Production': 'Universal Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Suicide Squad',
  'Year': '2016',
  'Rated': 'PG-13',
  'Released': '05 Aug 2016',
  'Runtime': '123 min',
  'Genre': 'Action, Adventure, Fantasy, Sci-Fi',
  'Director': 'David Ayer',
  'Writer': 'David Ayer',
  'Actors': 'Will Smith, Jaime FitzSimons, Ike Barinholtz, Margot Robbie',
  'Plot': 'A secret government agency recruits some of the most dangerous incarcerated super-villains to form a defensive task force. Their first mission: save the world from the apocalypse.',
  'Language': 'English, Japanese, Spanish',
  'Country': 'USA',
  'Awards': 'Won 1 Oscar. Another 16 wins & 39 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMjM1OTMxNzUyM15BMl5BanBnXkFtZTgwNjYzMTIzOTE@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.0/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '27%'},
   {'Source': 'Metacritic', 'Value': '40/100'}],
  'Metascore': '40',
  'imdbRating': '6.0',
  'imdbVotes': '565,160',
  'imdbID': 'tt1386697',
  'Type': 'movie',
  'DVD': '13 Dec 2016',
  'BoxOffice': '$325,021,779',
  'Production': 'Warner Bros. Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Halloween',
  'Year': '2018',
  'Rated': 'R',
  'Released': '19 Oct 2018',
  'Runtime': '106 min',
  'Genre': 'Horror, Thriller',
  'Director': 'David Gordon Green',
  'Writer': 'John Carpenter (based on characters created by), Debra Hill (based on characters created by), Jeff Fradley, Danny McBride, David Gordon Green',
  'Actors': 'Jamie Lee Curtis, Judy Greer, Andi Matichak, James Jude Courtney',
  'Plot': 'Laurie Strode confronts her long-time foe Michael Myers, the masked figure who has haunted her since she narrowly escaped his killing spree on Halloween night four decades ago.',
  'Language': 'English',
  'Country': 'USA',
  'Awards': '3 wins & 24 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMmMzNjJhYjUtNzFkZi00MWQ4LWJiMDEtYWM0NTAzNGZjMTI3XkEyXkFqcGdeQXVyOTE2OTMwNDk@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.6/10'},
   {'Source': 'Metacritic', 'Value': '67/100'}],
  'Metascore': '67',
  'imdbRating': '6.6',
  'imdbVotes': '105,585',
  'imdbID': 'tt1502407',
  'Type': 'movie',
  'DVD': 'N/A',
  'BoxOffice': 'N/A',
  'Production': 'The Weinstein Company',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Hidden Figures',
  'Year': '2016',
  'Rated': 'PG',
  'Released': '06 Jan 2017',
  'Runtime': '127 min',
  'Genre': 'Biography, Drama, History',
  'Director': 'Theodore Melfi',
  'Writer': 'Allison Schroeder (screenplay by), Theodore Melfi (screenplay by), Margot Lee Shetterly (based on the book by)',
  'Actors': 'Taraji P. Henson, Octavia Spencer, Janelle Monáe, Kevin Costner',
  'Plot': 'The story of a team of female African-American mathematicians who served a vital role in NASA during the early years of the U.S. space program.',
  'Language': 'English',
  'Country': 'USA',
  'Awards': 'Nominated for 3 Oscars. Another 37 wins & 85 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMzg2Mzg4YmUtNDdkNy00NWY1LWE3NmEtZWMwNGNlMzE5YzU3XkEyXkFqcGdeQXVyMjA5MTIzMjQ@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.8/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '93%'},
   {'Source': 'Metacritic', 'Value': '74/100'}],
  'Metascore': '74',
  'imdbRating': '7.8',
  'imdbVotes': '182,441',
  'imdbID': 'tt4846340',
  'Type': 'movie',
  'DVD': '11 Apr 2017',
  'BoxOffice': '$169,385,416',
  'Production': '20th Century Fox',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Crazy Rich Asians',
  'Year': '2018',
  'Rated': 'PG-13',
  'Released': '15 Aug 2018',
  'Runtime': '120 min',
  'Genre': 'Comedy, Drama, Romance',
  'Director': 'Jon M. Chu',
  'Writer': 'Peter Chiarelli (screenplay by), Adele Lim (screenplay by), Kevin Kwan (based on the novel "Crazy Rich Asians" by)',
  'Actors': 'Constance Wu, Henry Golding, Michelle Yeoh, Gemma Chan',
  'Plot': "This contemporary romantic comedy, based on a global bestseller, follows native New Yorker Rachel Chu to Singapore to meet her boyfriend's family.",
  'Language': 'English, Mandarin, Cantonese, Hokkien, French, Malay',
  'Country': 'USA',
  'Awards': 'Nominated for 2 Golden Globes. Another 12 wins & 60 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMTYxNDMyOTAxN15BMl5BanBnXkFtZTgwMDg1ODYzNTM@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.9/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '91%'},
   {'Source': 'Metacritic', 'Value': '74/100'}],
  'Metascore': '74',
  'imdbRating': '6.9',
  'imdbVotes': '119,244',
  'imdbID': 'tt3104988',
  'Type': 'movie',
  'DVD': '20 Nov 2018',
  'BoxOffice': 'N/A',
  'Production': 'Warner Bros. Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Thor: Ragnarok',
  'Year': '2017',
  'Rated': 'PG-13',
  'Released': '03 Nov 2017',
  'Runtime': '130 min',
  'Genre': 'Action, Adventure, Comedy, Fantasy, Sci-Fi',
  'Director': 'Taika Waititi',
  'Writer': 'Eric Pearson, Craig Kyle, Christopher L. Yost, Stan Lee (based on the Marvel comics by), Larry Lieber (based on the Marvel comics by), Jack Kirby (based on the Marvel comics by)',
  'Actors': 'Chris Hemsworth, Tom Hiddleston, Cate Blanchett, Idris Elba',
  'Plot': 'Imprisoned on the planet Sakaar, Thor must race against time to return to Asgard and stop Ragnarök, the destruction of his world, at the hands of the powerful and ruthless villain Hela.',
  'Language': 'English',
  'Country': 'USA',
  'Awards': '6 wins & 48 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMjMyNDkzMzI1OF5BMl5BanBnXkFtZTgwODcxODg5MjI@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.9/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '93%'},
   {'Source': 'Metacritic', 'Value': '74/100'}],
  'Metascore': '74',
  'imdbRating': '7.9',
  'imdbVotes': '532,010',
  'imdbID': 'tt3501632',
  'Type': 'movie',
  'DVD': '06 Mar 2018',
  'BoxOffice': '$314,971,245',
  'Production': 'Walt Disney Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Straight Outta Compton',
  'Year': '2015',
  'Rated': 'R',
  'Released': '14 Aug 2015',
  'Runtime': '147 min',
  'Genre': 'Biography, Drama, History, Music',
  'Director': 'F. Gary Gray',
  'Writer': 'Jonathan Herman (screenplay), Andrea Berloff (screenplay), S. Leigh Savidge (story), Alan Wenkus (story), Andrea Berloff (story)',
  'Actors': "O'Shea Jackson Jr., Corey Hawkins, Jason Mitchell, Neil Brown Jr.",
  'Plot': 'The rap group NWA emerges from the mean streets of Compton in Los Angeles, California, in the mid-1980s and revolutionizes Hip Hop culture with their music and tales about life in the hood.',
  'Language': 'English',
  'Country': 'USA',
  'Awards': 'Nominated for 1 Oscar. Another 28 wins & 39 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMTA5MzkyMzIxNjJeQTJeQWpwZ15BbWU4MDU0MDk0OTUx._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.9/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '88%'},
   {'Source': 'Metacritic', 'Value': '72/100'}],
  'Metascore': '72',
  'imdbRating': '7.9',
  'imdbVotes': '171,104',
  'imdbID': 'tt1398426',
  'Type': 'movie',
  'DVD': '19 Jan 2016',
  'BoxOffice': '$129,402,415',
  'Production': 'Universal Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Split',
  'Year': '2016',
  'Rated': 'PG-13',
  'Released': '20 Jan 2017',
  'Runtime': '117 min',
  'Genre': 'Horror, Thriller',
  'Director': 'M. Night Shyamalan',
  'Writer': 'M. Night Shyamalan',
  'Actors': 'James McAvoy, Anya Taylor-Joy, Betty Buckley, Haley Lu Richardson',
  'Plot': 'Three girls are kidnapped by a man with a diagnosed 23 distinct personalities. They must try to escape before the apparent emergence of a frightful new 24th.',
  'Language': 'English',
  'Country': 'USA, Japan',
  'Awards': '9 wins & 21 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BZTJiNGM2NjItNDRiYy00ZjY0LTgwNTItZDBmZGRlODQ4YThkL2ltYWdlXkEyXkFqcGdeQXVyMjY5ODI4NDk@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.3/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '77%'},
   {'Source': 'Metacritic', 'Value': '62/100'}],
  'Metascore': '62',
  'imdbRating': '7.3',
  'imdbVotes': '391,074',
  'imdbID': 'tt4972582',
  'Type': 'movie',
  'DVD': '18 Apr 2017',
  'BoxOffice': '$138,120,085',
  'Production': 'Universal Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'La La Land',
  'Year': '2016',
  'Rated': 'PG-13',
  'Released': '25 Dec 2016',
  'Runtime': '128 min',
  'Genre': 'Comedy, Drama, Music, Musical, Romance',
  'Director': 'Damien Chazelle',
  'Writer': 'Damien Chazelle',
  'Actors': 'Ryan Gosling, Emma Stone, Amiée Conn, Terry Walters',
  'Plot': 'While navigating their careers in Los Angeles, a pianist and an actress fall in love while attempting to reconcile their aspirations for the future.',
  'Language': 'English',
  'Country': 'USA, Hong Kong',
  'Awards': 'Won 6 Oscars. Another 235 wins & 276 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMzUzNDM2NzM2MV5BMl5BanBnXkFtZTgwNTM3NTg4OTE@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.0/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '91%'},
   {'Source': 'Metacritic', 'Value': '93/100'}],
  'Metascore': '93',
  'imdbRating': '8.0',
  'imdbVotes': '463,791',
  'imdbID': 'tt3783958',
  'Type': 'movie',
  'DVD': '25 Apr 2017',
  'BoxOffice': '$151,058,124',
  'Production': 'Liongate Films',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Avengers: Age of Ultron',
  'Year': '2015',
  'Rated': 'PG-13',
  'Released': '01 May 2015',
  'Runtime': '141 min',
  'Genre': 'Action, Adventure, Sci-Fi',
  'Director': 'Joss Whedon',
  'Writer': 'Joss Whedon, Stan Lee (based on the Marvel comics by), Jack Kirby (based on the Marvel comics by), Joe Simon (character created by: Captain America), Jack Kirby (character created by: Captain America), Jim Starlin (character created by: Thanos)',
  'Actors': 'Robert Downey Jr., Chris Hemsworth, Mark Ruffalo, Chris Evans',
  'Plot': "When Tony Stark and Bruce Banner try to jump-start a dormant peacekeeping program called Ultron, things go horribly wrong and it's up to Earth's mightiest heroes to stop the villainous Ultron from enacting his terrible plan.",
  'Language': 'English, Korean',
  'Country': 'USA',
  'Awards': '8 wins & 47 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMTM4OGJmNWMtOTM4Ni00NTE3LTg3MDItZmQxYjc4N2JhNmUxXkEyXkFqcGdeQXVyNTgzMDMzMTg@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.3/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '75%'},
   {'Source': 'Metacritic', 'Value': '66/100'}],
  'Metascore': '66',
  'imdbRating': '7.3',
  'imdbVotes': '700,903',
  'imdbID': 'tt2395427',
  'Type': 'movie',
  'DVD': '02 Oct 2015',
  'BoxOffice': '$429,113,729',
  'Production': 'Walt Disney Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'The Hunger Games: Mockingjay - Part 2',
  'Year': '2015',
  'Rated': 'PG-13',
  'Released': '20 Nov 2015',
  'Runtime': '137 min',
  'Genre': 'Action, Adventure, Sci-Fi, Thriller',
  'Director': 'Francis Lawrence',
  'Writer': 'Peter Craig (screenplay), Danny Strong (screenplay), Suzanne Collins (adaptation), Suzanne Collins (novel)',
  'Actors': 'Jennifer Lawrence, Josh Hutcherson, Liam Hemsworth, Woody Harrelson',
  'Plot': 'Katniss and a team of rebels from District 13 prepare for the final battle that will decide the fate of Panem.',
  'Language': 'English',
  'Country': 'USA',
  'Awards': '16 wins & 34 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BNjQzNDI2NTU1Ml5BMl5BanBnXkFtZTgwNTAyMDQ5NjE@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.5/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '70%'},
   {'Source': 'Metacritic', 'Value': '65/100'}],
  'Metascore': '65',
  'imdbRating': '6.5',
  'imdbVotes': '265,897',
  'imdbID': 'tt1951266',
  'Type': 'movie',
  'DVD': '22 Mar 2016',
  'BoxOffice': '$254,641,329',
  'Production': 'Lionsgate Films',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'The Martian',
  'Year': '2015',
  'Rated': 'PG-13',
  'Released': '02 Oct 2015',
  'Runtime': '144 min',
  'Genre': 'Adventure, Drama, Sci-Fi',
  'Director': 'Ridley Scott',
  'Writer': 'Drew Goddard (screenplay by), Andy Weir (based on the novel by)',
  'Actors': 'Matt Damon, Jessica Chastain, Kristen Wiig, Jeff Daniels',
  'Plot': 'An astronaut becomes stranded on Mars after his team assume him dead, and must rely on his ingenuity to find a way to signal to Earth that he is alive.',
  'Language': 'English, Mandarin',
  'Country': 'UK, USA, Hungary',
  'Awards': 'Nominated for 7 Oscars. Another 40 wins & 188 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMTc2MTQ3MDA1Nl5BMl5BanBnXkFtZTgwODA3OTI4NjE@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.0/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '91%'},
   {'Source': 'Metacritic', 'Value': '80/100'}],
  'Metascore': '80',
  'imdbRating': '8.0',
  'imdbVotes': '714,155',
  'imdbID': 'tt3659388',
  'Type': 'movie',
  'DVD': '12 Jan 2016',
  'BoxOffice': '$202,313,768',
  'Production': '20th Century Fox',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Wonder',
  'Year': '2017',
  'Rated': 'PG',
  'Released': '17 Nov 2017',
  'Runtime': '113 min',
  'Genre': 'Drama, Family',
  'Director': 'Stephen Chbosky',
  'Writer': 'Stephen Chbosky (screenplay by), Steve Conrad (screenplay by), Jack Thorne (screenplay by), R.J. Palacio (based on the novel by)',
  'Actors': 'Jacob Tremblay, Owen Wilson, Izabela Vidovic, Julia Roberts',
  'Plot': 'Based on the New York Times bestseller, this movie tells the incredibly inspiring and heartwarming story of August Pullman, a boy with facial differences who enters the fifth grade, attending a mainstream elementary school for the first time.',
  'Language': 'English',
  'Country': 'USA, Hong Kong',
  'Awards': 'Nominated for 1 Oscar. Another 4 wins & 26 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BYjFhOWY0OTgtNDkzMC00YWJkLTk1NGEtYWUxNjhmMmQ5ZjYyXkEyXkFqcGdeQXVyMjMxOTE0ODA@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.0/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '85%'},
   {'Source': 'Metacritic', 'Value': '66/100'}],
  'Metascore': '66',
  'imdbRating': '8.0',
  'imdbVotes': '123,097',
  'imdbID': 'tt2543472',
  'Type': 'movie',
  'DVD': '13 Feb 2018',
  'BoxOffice': '$132,090,170',
  'Production': 'Lionsgate',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Cinderella',
  'Year': '2015',
  'Rated': 'PG',
  'Released': '13 Mar 2015',
  'Runtime': '105 min',
  'Genre': 'Drama, Family, Fantasy, Romance',
  'Director': 'Kenneth Branagh',
  'Writer': 'Chris Weitz (screenplay by), Charles Perrault (story)',
  'Actors': 'Cate Blanchett, Lily James, Richard Madden, Helena Bonham Carter',
  'Plot': "When her father unexpectedly dies, young Ella finds herself at the mercy of her cruel stepmother and her scheming stepsisters. Never one to give up hope, Ella's fortunes begin to change after meeting a dashing stranger.",
  'Language': 'English',
  'Country': 'USA, UK',
  'Awards': 'Nominated for 1 Oscar. Another 8 wins & 36 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMjMxODYyODEzN15BMl5BanBnXkFtZTgwMDk4OTU0MzE@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.9/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '84%'},
   {'Source': 'Metacritic', 'Value': '67/100'}],
  'Metascore': '67',
  'imdbRating': '6.9',
  'imdbVotes': '150,514',
  'imdbID': 'tt1661199',
  'Type': 'movie',
  'DVD': '15 Sep 2015',
  'BoxOffice': '$183,327,144',
  'Production': 'Walt Disney Pictures',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'Hotel Transylvania 3: Summer Vacation',
  'Year': '2018',
  'Rated': 'PG',
  'Released': '13 Jul 2018',
  'Runtime': '97 min',
  'Genre': 'Animation, Adventure, Comedy, Family, Fantasy, Horror',
  'Director': 'Genndy Tartakovsky',
  'Writer': 'Genndy Tartakovsky, Michael McCullers',
  'Actors': 'Adam Sandler, Andy Samberg, Selena Gomez, Kevin James',
  'Plot': 'Count Dracula and company participate in a cruise for sea-loving monsters, unaware that their boat is being commandeered by the monster-hating Van Helsing family.',
  'Language': 'English, Spanish, German, Russian',
  'Country': 'USA, South Korea',
  'Awards': '2 wins & 8 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BNjA1MzU5MTY3OF5BMl5BanBnXkFtZTgwNTU5MDA3NTM@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.3/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '62%'},
   {'Source': 'Metacritic', 'Value': '54/100'}],
  'Metascore': '54',
  'imdbRating': '6.3',
  'imdbVotes': '49,082',
  'imdbID': 'tt5220122',
  'Type': 'movie',
  'DVD': '25 Sep 2018',
  'BoxOffice': 'N/A',
  'Production': 'Columbia Pictures / Sony Pictures Animation',
  'Website': 'N/A',
  'Response': 'True'},
 {'Title': 'John Wick: Chapter 3 - Parabellum',
  'Year': '2019',
  'Rated': 'R',
  'Released': '17 May 2019',
  'Runtime': '131 min',
  'Genre': 'Action, Crime, Thriller',
  'Director': 'Chad Stahelski',
  'Writer': 'Derek Kolstad (screenplay by), Shay Hatten (screenplay by), Chris Collins (screenplay by), Marc Abrams (screenplay by), Derek Kolstad (story by), Derek Kolstad (based on characters created by)',
  'Actors': 'Keanu Reeves, Halle Berry, Ian McShane, Laurence Fishburne',
  'Plot': "John Wick is on the run after killing a member of the international assassins' guild, and with a $14 million price tag on his head, he is the target of hit men and women everywhere.",
  'Language': 'English, Russian, Japanese, Indonesian, Mandarin, Italian, Arabic, Latin',
  'Country': 'USA',
  'Awards': '12 wins & 24 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMDg2YzI0ODctYjliMy00NTU0LTkxODYtYTNkNjQwMzVmOTcxXkEyXkFqcGdeQXVyNjg2NjQwMDQ@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.5/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '90%'},
   {'Source': 'Metacritic', 'Value': '73/100'}],
  'Metascore': '73',
  'imdbRating': '7.5',
  'imdbVotes': '221,237',
  'imdbID': 'tt6146586',
  'Type': 'movie',
  'DVD': '23 Aug 2019',
  'BoxOffice': 'N/A',
  'Production': 'Lionsgate',
  'Website': 'N/A',
  'Response': 'True'}]

In [23]:
api[35] = {'Title': 'Halloween',
 'Year': '2018',
 'Rated': 'R',
 'Released': '19 Oct 2018',
 'Runtime': '106 min',
 'Genre': 'Horror, Thriller',
 'Actors': 'Jamie Lee Curtis, Judy Greer, Andi Matichak, James Jude Courtney',
 'Language': 'English',
 'Country': 'USA',
 'Awards': '3 wins & 24 nominations.',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.6/10'},
  {'Source': 'Rotten Tomatoes', 'Value': ''},
  {'Source': 'Metacritic', 'Value': '67/100'}],
 'Metascore': '67',
 'imdbRating': '6.6',
 'imdbVotes': '105,585',
 'imdbID': 'tt1502407',
 'BoxOffice': 'N/A',
 'Production': 'The Weinstein Company'}

In [24]:
api[47] = {'Title': 'Cinderella',
 'Year': '2015',
 'Rated': 'PG',
 'Released': '13 March 2015',
 'Runtime': '113 min',
 'Genre': 'Fantasy',
 'Actors': 'Rupesh Bane, Adiputra Suryadinata, Aditya Nakti, Amar Padwal',
 'Language': 'Hindi',
 'Country': 'India',
 'Awards': 'N/A',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': ''},
            {'Source': 'Rotten Tomatoes', 'Value': ''},
            {'Source': 'Metacritic', 'Value': ''}],
 'Metascore': 'N/A',
 'imdbRating': '6.9',
 'imdbVotes': 'N/A',
 'imdbID': 'tt5089556',
 'BoxOffice': '$183,327,144',
 'Production': 'Walt Disney Pictures'}

In [25]:
#create api dataframe, strip leading,trailing spaces
api_df = pd.DataFrame.from_dict(api)
api_df.applymap(lambda c: c.strip() if isinstance(c, str) else c)

#select(via drop) columns, create combo id
api_df2 = api_df.drop(['Director', 'Actors', 'Plot', 'Type', 'Website', 'Response', 'DVD'], axis=1)
api_df2['combo_id'] = api_df2['Year'] + ' ' + api_df2['Title']

In [26]:
#join all dataframes
api_b_t = api_df2.join(b_and_t,how='left')
api_b_t['release_month'] = api_b_t['release_date'].apply(lambda m: m.month)
api_b_t['release_month'] = api_b_t['release_month'].apply(lambda x: calendar.month_abbr[x])
api_b_t

,Title,Year,Rated,Released,Runtime,Genre,Writer,Language,Country,Awards,...,combo_id,budgets_movie,release_date,production_budget,domestic_gross,domestic_profit,titles_movie,genres,tconst,release_month
0,Star Wars: Episode VII - The Force Awakens,2015,PG-13,18 Dec 2015,138 min,"Action, Adventure, Sci-Fi","Lawrence Kasdan, J.J. Abrams, Michael Arndt, G...",English,USA,Nominated for 5 Oscars. Another 63 wins & 125 ...,...,2015 Star Wars: Episode VII - The Force Awakens,Star Wars Ep. VII: The Force Awakens,2015-12-18,306000000,936662225,630662225,NaN,"Action,Adventure,Fantasy",tt2527338,Dec
1,Black Panther,2018,PG-13,16 Feb 2018,134 min,"Action, Adventure, Sci-Fi","Ryan Coogler, Joe Robert Cole, Stan Lee (based...","English, Swahili, Nama, Xhosa, Korean",USA,Won 3 Oscars. Another 108 wins & 266 nominations.,...,2018 Black Panther,Black Panther,2018-02-16,200000000,700059566,500059566,Black Panther,"Action,Adventure,Sci-Fi",tt1825683,Feb
2,Jurassic World,2015,PG-13,12 Jun 2015,124 min,"Action, Adventure, Sci-Fi","Rick Jaffa (screenplay by), Amanda Silver (scr...",English,USA,15 wins & 57 nominations.,...,2015 Jurassic World,Jurassic World,2015-06-12,215000000,652270625,437270625,Jurassic World,"Action,Adventure,Sci-Fi",tt0369610,Jun
3,Incredibles 2,2018,PG,15 Jun 2018,118 min,"Animation, Action, Adventure, Comedy, Family, ...",Brad Bird,English,USA,Nominated for 1 Oscar. Another 12 wins & 72 no...,...,2018 Incredibles 2,Incredibles 2,2018-06-15,200000000,608581744,408581744,Incredibles 2,"Action,Adventure,Animation",tt3606756,Jun
4,Avengers: Infinity War,2018,PG-13,27 Apr 2018,149 min,"Action, Adventure, Sci-Fi","Christopher Markus (screenplay by), Stephen Mc...",English,USA,Nominated for 1 Oscar. Another 44 wins & 70 no...,...,2018 Avengers: Infinity War,Avengers: Infinity War,2018-04-27,300000000,678815482,378815482,Avengers: Infinity War,"Action,Adventure,Sci-Fi",tt4154756,Apr
5,Beauty and the Beast,2017,PG,17 Mar 2017,129 min,"Family, Fantasy, Musical, Romance","Stephen Chbosky (screenplay by), Evan Spilioto...",English,USA,Nominated for 2 Oscars. Another 15 wins & 71 n...,...,2017 Beauty and the Beast,Beauty and the Beast,2017-03-17,160000000,504014165,344014165,Beauty and the Beast,"Family,Fantasy,Musical",tt2771200,Mar
6,Rogue One: A Star Wars Story,2016,PG-13,16 Dec 2016,133 min,"Action, Adventure, Sci-Fi","Chris Weitz (screenplay by), Tony Gilroy (scre...",English,USA,Nominated for 2 Oscars. Another 24 wins & 80 n...,...,2016 Rogue One: A Star Wars Story,Rogue One: A Star Wars Story,2016-12-16,200000000,532177324,332177324,Rogue One: A Star Wars Story,"Action,Adventure,Sci-Fi",tt3748528,Dec
7,Jumanji: Welcome to the Jungle,2017,PG-13,20 Dec 2017,119 min,"Action, Adventure, Comedy, Fantasy","Chris McKenna (screenplay by), Erik Sommers (s...",English,"USA, India, Canada, UK, Australia, Germany",5 wins & 14 nominations.,...,2017 Jumanji: Welcome to the Jungle,Jumanji: Welcome to the Jungle,2017-12-20,90000000,404508916,314508916,Jumanji: Welcome to the Jungle,"Action,Adventure,Comedy",tt2283362,Dec
8,Deadpool,2016,R,12 Feb 2016,108 min,"Action, Adventure, Comedy, Sci-Fi","Rhett Reese, Paul Wernick",English,USA,Nominated for 2 Golden Globes. Another 28 wins...,...,2016 Deadpool,Deadpool,2016-02-12,58000000,363070709,305070709,Deadpool,"Action,Adventure,Comedy",tt1431045,Feb
9,Star Wars: Episode VIII - The Last Jedi,2017,PG-13,15 Dec 2017,152 min,"Action, Adventure, Fantasy, Sci-Fi","Rian Johnson, George Lucas (based on character...",English,USA,Nominated for 4 Oscars. Another 23 wins & 89 n...,...,2017 Star Wars: Episode VIII - The Last Jedi,Star Wars Ep. VIII: The Last Jedi,2017-12-15,317000000,620181382,303181382,NaN,"Action,Adventure,Fantasy",tt2527336,Dec


In [27]:
#create dataframes for further analysis
profit_by_mpaa = pd.pivot_table(api_b_t, values='domestic_profit', columns=['Rated'], aggfunc=np.average)
imdb_score = pd.pivot_table(api_b_t, values=['domestic_profit'], columns=['imdbRating'], aggfunc=np.average)
profit_by_month = pd.pivot_table(api_b_t, values='domestic_profit', columns=['release_month'], aggfunc=np.average)
profit_by_month = profit_by_month[["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]]

In [28]:
#select api_b_t columns and bring to final dataframe
movies = api_b_t[['Title','Year','release_month','Rated','Runtime','Genre','Metascore','imdbRating','imdbVotes','imdbID','production_budget','domestic_gross','domestic_profit','genres']]

#split genres for deeper analysis
movies[['primary_genre','secondary_genre','tertiary_genre']] = movies.genres.str.split(',', expand=True) 

/Users/algrhythm/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [29]:
#create and populate movie type (action/adventure, not action/adventure) based on split genres (uses '.isin', 'np.logical_and','.replace')
actadv_valid = pd.DataFrame({'col': ['Action', 'Adventure']}) #a dictionary for our film types
movies['actionadv_primary'] = movies.primary_genre.isin(actadv_valid.col)
movies['actionadv_secondary'] = movies.secondary_genre.isin(actadv_valid.col)
movies['film_type'] = np.logical_and(movies['actionadv_primary'], movies['actionadv_secondary'])
movies['film_type'] = movies['film_type'].replace({True: 'Action/Adventure', False: 'Not Action/Adventure'})
#movies['release_month'] = movies['release_month'].apply(lambda x: calendar.month_abbr[x])

/Users/algrhythm/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/algrhythm/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/algrhythm/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set

In [30]:
#select columns for final, cleaned dataframe
movies = movies[['Title', 'Year', 'release_month', 'Rated', 'Runtime',
       'Metascore', 'imdbRating', 'imdbVotes', 'production_budget',
       'domestic_gross', 'domestic_profit','genres','primary_genre', 'secondary_genre',
       'tertiary_genre','film_type','imdbID']]

### View and export cleaned data

In [31]:
movies

,Title,Year,release_month,Rated,Runtime,Metascore,imdbRating,imdbVotes,production_budget,domestic_gross,domestic_profit,genres,primary_genre,secondary_genre,tertiary_genre,film_type,imdbID
0,Star Wars: Episode VII - The Force Awakens,2015,Dec,PG-13,138 min,81,7.9,"826,625",306000000,936662225,630662225,"Action,Adventure,Fantasy",Action,Adventure,Fantasy,Action/Adventure,tt2488496
1,Black Panther,2018,Feb,PG-13,134 min,88,7.3,"565,951",200000000,700059566,500059566,"Action,Adventure,Sci-Fi",Action,Adventure,Sci-Fi,Action/Adventure,tt1825683
2,Jurassic World,2015,Jun,PG-13,124 min,59,7.0,"556,312",215000000,652270625,437270625,"Action,Adventure,Sci-Fi",Action,Adventure,Sci-Fi,Action/Adventure,tt0369610
3,Incredibles 2,2018,Jun,PG,118 min,80,7.6,"225,556",200000000,608581744,408581744,"Action,Adventure,Animation",Action,Adventure,Animation,Action/Adventure,tt3606756
4,Avengers: Infinity War,2018,Apr,PG-13,149 min,68,8.5,"748,996",300000000,678815482,378815482,"Action,Adventure,Sci-Fi",Action,Adventure,Sci-Fi,Action/Adventure,tt4154756
5,Beauty and the Beast,2017,Mar,PG,129 min,65,7.1,"253,795",160000000,504014165,344014165,"Family,Fantasy,Musical",Family,Fantasy,Musical,Not Action/Adventure,tt2771200
6,Rogue One: A Star Wars Story,2016,Dec,PG-13,133 min,65,7.8,"519,590",200000000,532177324,332177324,"Action,Adventure,Sci-Fi",Action,Adventure,Sci-Fi,Action/Adventure,tt3748528
7,Jumanji: Welcome to the Jungle,2017,Dec,PG-13,119 min,58,6.9,"284,295",90000000,404508916,314508916,"Action,Adventure,Comedy",Action,Adventure,Comedy,Action/Adventure,tt2283362
8,Deadpool,2016,Feb,R,108 min,65,8.0,"857,421",58000000,363070709,305070709,"Action,Adventure,Comedy",Action,Adventure,Comedy,Action/Adventure,tt1431045
9,Star Wars: Episode VIII - The Last Jedi,2017,Dec,PG-13,152 min,85,7.0,"524,507",317000000,620181382,303181382,"Action,Adventure,Fantasy",Action,Adventure,Fantasy,Action/Adventure,tt2527336


In [32]:
profit_by_month

release_month,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
domestic_profit,133141585.0,3.253903e+08,1.923530e+08,225461996.5,169238256.6,2.876453e+08,204282833.0,1.426103e+08,292481748.0,1.496882e+08,145225857.8,2.783287e+08


In [33]:
profit_by_mpaa

Rated,PG,PG-13,R
domestic_profit,2.137306e+08,2.362270e+08,185262059.9


In [34]:
imdb_score

imdbRating,6.2,6.3,6.4,6.5,6.6,6.9,7.0,7.1,7.2,7.3,7.4,7.5,7.6,7.7,7.8,7.9,8.0,8.2,8.5
domestic_profit,247719760.0,1.637989e+08,208671000.0,207554116.0,149721034.5,194345214.4,3.051713e+08,269671605.0,163007020.0,268076865.6,203588557.0,136762226.5,299197422.5,1.883071e+08,2.116230e+08,299639433.0,1.702668e+08,181461711.0,378815482.0


Export all data to csv

In [35]:
movies.to_csv('movies')
profit_by_mpaa.to_csv('domestic_profit_by_rating')
imdb_score.to_csv('domestic_profit_by_user_rating')
profit_by_month.to_csv('domestic_profit_by_month')